In [68]:
%reset -f
import networkx as nx
import sqlite3
import os
import warnings
import json
import pandas as pd
import tweepy

warnings.simplefilter(action='ignore', category=FutureWarning)


In [69]:
# Load Twitter credentials
api_keys = json.load(open('twitter-api-keys.local.json'))
bearer_token = api_keys["bearer_token"]
client = tweepy.Client(
    bearer_token=api_keys['bearer_token'], wait_on_rate_limit=True)

auth = tweepy.OAuthHandler(
    consumer_key=api_keys['api_key'],
    consumer_secret=api_keys['api_key_secret'],
    access_token=api_keys['access_token'],
    access_token_secret=api_keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)


In [70]:
# Create databases and tables
conn = sqlite3.connect('twitter.db')
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS users
                (id INTEGER PRIMARY KEY NOT NULL,
                screen_name TEXT NOT NULL,
                followers_count INTEGER,
                following_count INTEGER,
                verified INTEGER,
                followers TEXT DEFALUT '',
                friends TEXT DEFALUT '',
                mutuals TEXT DEFALUT '')''')

In [64]:
# # delete database
# c.execute('''DROP TABLE IF EXISTS users''')
# conn.commit()

In [71]:
me = 'fcx_xm'
me_info = api.get_user(screen_name=me)
c.execute('''INSERT OR IGNORE INTO users
                (id, screen_name, followers_count, following_count, verified)
                VALUES (?, ?, ?, ?, ?)''',
                (me_info.id, me_info.screen_name, me_info.followers_count,
                me_info.friends_count, me_info.verified))
conn.commit()

In [73]:
def get_followers(screen_name):
    print('Getting followers')
    if c.execute('''SELECT followers FROM users WHERE screen_name = ?''', (screen_name,)).fetchone()[0] != None:
        print('- Already have friends for ' + screen_name)
        return 

    ids = []
    for fid in tweepy.Cursor(api.get_friend_ids, screen_name=screen_name, count=5000).items():
        ids.append(fid)
    
    info = []
    for i in range(0, len(ids), 100):
        try:
            chunk = ids[i:i+100]
            print('- Getting friends info for chunk ' + str(i) + ' to ' + str(i+100) + ' of ' + str(len(ids)) + ' for ' + screen_name)
            info.extend(api.lookup_users(user_id=chunk))
        except:
            import traceback
            traceback.print_exc()
            print('Something went wrong, skipping...')

    followers_list = {}
    for i in info:
        followers_list[i.id] = i.screen_name
    c.execute('''UPDATE users SET followers = ? WHERE screen_name = ?''',
                (json.dumps(followers_list), screen_name))
    conn.commit()


def get_friends(screen_name):
    print('Getting friends')

    if c.execute('''SELECT friends FROM users WHERE screen_name = ?''', (screen_name,)).fetchone()[0] != None:
        print('- Already have friends for ' + screen_name)
        return 

    ids = []
    for fid in tweepy.Cursor(api.get_friend_ids, screen_name=screen_name, count=5000).items():
        ids.append(fid)
    
    info = []
    for i in range(0, len(ids), 100):
        try:
            chunk = ids[i:i+100]
            # print chunk and len of ids
            print('- Getting friends info for chunk ' + str(i) + ' to ' + str(i+100) + ' of ' + str(len(ids)) + ' for ' + screen_name)
            info.extend(api.lookup_users(user_id=chunk))
        except:
            import traceback
            traceback.print_exc()
            print('Something went wrong, skipping...')

    friends_list = {}
    for i in info:
        friends_list[i.id] = i.screen_name

    c.execute('''UPDATE users SET friends = ? WHERE screen_name = ?''',
                (json.dumps(friends_list), screen_name))
    conn.commit()

def get_mutuals(screen_name):
    get_followers(screen_name)
    followers = json.loads(c.execute('''SELECT followers FROM users WHERE screen_name = ?''', (screen_name,)).fetchone()[0])
    get_friends(screen_name)
    friends = json.loads(c.execute('''SELECT friends FROM users WHERE screen_name = ?''', (screen_name,)).fetchone()[0])
    # save in mutuals dict of id: screename those who are in both followers and friends
    mutuals = followers.keys() & friends.keys()
    mutuals = {k: friends[k] for k in mutuals}
    c.execute('''UPDATE users SET mutuals = ? WHERE screen_name = ?''',
                (json.dumps(mutuals), screen_name))
    conn.commit()

Getting followers
- Already have friends for fcx_xm
Getting friends
- Already have friends for fcx_xm


In [193]:
def build_mutuals_df(screen_name):
    global df
    mutuals_df = pd.DataFrame(columns=['id', 'screen_name'])
    mutuals_json = json.loads(
        df.loc[df['screen_name'] == screen_name, 'mutuals'].values[0])
    for k, v in mutuals_json.items():
        mutuals_df = mutuals_df.append(
            {'id': int(k), 'screen_name': v}, ignore_index=True)
    # Merge left join
    mutuals_df = mutuals_df.merge(df, on='id', how='left').drop(mutuals_df.columns[mutuals_df.columns.str.endswith(
        '_y')], axis=1).rename(columns=lambda x: x[:-2] if x.endswith('_x') else x)
    return mutuals_df


In [194]:
def get_filtered_users(mutuals_df, rules, exclude):
    global df
    mutuals_matching = []
    verified = []
    min_followers = []
    for index, row in mutuals_df.iterrows():
        if row['screen_name'] not in exclude and 'verified' in rules and row['verified'] == rules['verified']:
            verified.append(row['id'])
        # check min_followers
        if row['screen_name'] not in exclude and 'min_followers' in rules and row['followers_count'] >= rules['min_followers']:
            min_followers.append(row['id'])

    mutuals_matching = list(set(verified) & set(min_followers))
    mutuals_matching_df = mutuals_df.loc[mutuals_df['id'].isin(
        mutuals_matching)]
    return mutuals_matching_df


In [195]:
def save_path(path, rules):
    filename = 'built_paths.json'
    # check if file exists
    if os.path.isfile(filename):
        with open(filename, 'r') as f:
            data = json.load(f)
    else:
        data = {}

    # Stringify rules json
    rules = json.dumps(rules)

    obj = {
        'rules': rules,
        'dst': path[-1],
        'path': path
    }

    # Check if data[rules] exists and add obj to it if it does not exist.
    # Check
    if rules in data:
        if obj not in data[path[0]]:
            data[path[0]] = obj
    else:
        data[path[0]] = [obj]

    with open(filename, 'w') as f:
        json.dump(data, f)


def load_paths():
    filename = 'built_paths.json'
    if os.path.isfile(filename):
        with open(filename, 'r') as f:
            data = json.load(f)
    else:
        data = {}
    return data


In [198]:
def shortest_path_to_celeb(exclude):
    global df
    print('Getting shortest path to celeb')
    path = []
    max_depth = 3
    rules = {'verified': True, 'min_followers': 1e5}
    found = False
    i = 0
    # Starting with me
    current_user = me
    path.append(current_user)
    while ((len(path) <= max_depth) and (not found)):
        i += 1
        print('----------------- Iteration ' + str(i) + ' -----------------')
        print('Current user: {}'.format(current_user))
        get_mutuals(current_user)
        mutuals_df = build_mutuals_df(current_user)

        mutuals_matching = get_filtered_users(mutuals_df, rules, exclude)

        # TODO: add backtracking when max_depth is reached and no match is found
        if len(mutuals_matching) == 0:
            print('No matching users found')
            mutuals_df = mutuals_df.loc[~mutuals_df['screen_name'].isin(
                exclude)]
            current_user = mutuals_df.sort_values(
                'followers_count', ascending=False).iloc[0]['screen_name']
            path.append(current_user)
        else:
            most_followers = mutuals_matching.sort_values(
                'followers_count', ascending=False).iloc[0]
            path.append(most_followers['screen_name'])

            print('Matching user with most followers: {}, with {} followers'.format(
                most_followers['screen_name'], most_followers['followers_count']))

            found = True

        print('Built path: {}'.format(path))

    save_path(path, rules)
    return path


x = shortest_path_to_celeb(['vodafone_es'])


Getting shortest path to celeb
----------------- Iteration 1 -----------------
Current user: fcx_xm
Getting followers
- Already have followers for fcx_xm
Getting friends
- Already have friends for fcx_xm
No matching users found
Built path: ['fcx_xm', 'Meescapo']
----------------- Iteration 2 -----------------
Current user: Meescapo
Getting followers
- Already have followers for Meescapo
Getting friends
- Already have friends for Meescapo
Matching user with most followers: Loulogio_Pi, with 310410 followers
Built path: ['fcx_xm', 'Meescapo', 'Loulogio_Pi']


In [197]:
max_paths = 3
n_paths = 0
exclude_users = ['vodafone_es']

# Load paths
paths = load_paths()

if me in paths:
    for path in paths[me]:
        print(path)
